In [ ]:
import os
import argparse
import numpy as np
import SimpleITK as sitk
from itkwidgets import view
import pymskt as mskt
from pymskt.mesh import BoneMesh, CartilageMesh, meshTools
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import json
import pandas as pd
import datetime
import time
import torch
from pymskt.mesh.meshTransform import get_linear_transform_matrix
import sys
import gc
import pyvista as pv
import itkwidgets
import vtk
import pickle
from NSM.models.triplanar import TriplanarDecoder
from NSM.reconstruct import reconstruct_mesh
from NSM.mesh.interpolate import interpolate_points, interpolate_mesh, interpolate_common
from NSM.mesh import create_mesh
sys.path.append('/dataNAS/people/kenziew/projects/knee_pipeline/DEPENDENCIES/nsosim')
# sys.path.append('/dataNAS/people/kenziew/projects/pymskt-test/pymskt')
# sys.path.append('/dataNAS/people/kenziew/projects/pymskt-test/nsm')
from nsosim.nsm_fitting import apply_transform, undo_transform
from nsosim.utils import load_model
from scipy.spatial import cKDTree
import ast
import numpy as np
import pyvista as pv
from vtk.util.numpy_support import numpy_to_vtk

[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc: No such file or directory
collect2: error: ld returned 1 exit status

OK


In [ ]:
def list_all_directories(path, recursive=True):
    """
    List all directories in the given path.

    Parameters:
    - path (str): The path to search for directories.
    - recursive (bool): If True, list directories recursively. If False, list only top-level directories.

    Returns:
    - List of directory paths.
    """
    directories = []
    if recursive:
        for root, dirs, files in os.walk(path):
            for dir_name in dirs:
                dir_path = os.path.join(root, dir_name)
                directories.append(dir_path)
    else:
        # List only top-level directories
        with os.scandir(path) as entries:
            for entry in entries:
                if entry.is_dir():
                    directories.append(entry.path)
    return directories

def list_all_files(path, extension=None):
    files = []
    for root, dirs, filenames in os.walk(path):
        for filename in filenames:
            if extension is None or filename.endswith(extension):
                file_path = os.path.join(root, filename)
                files.append(file_path)
    return files

def masked_healthy_mesh(
    mesh,
    threshold,
    percent_scalar='percent_people_with_cartilage',
    label_in='most_likely_label',
    prob_in='most_likely_prob',
    label_out='label',                        # what update_sub_labels expects
    prob_out='most_likely_prob_masked',
    percent_out='percent_people_with_cartilage_masked',
    keep_only=('percent_people_with_cartilage',),  # None = keep everything
    subset=False                               # True -> drop <thr points via threshold
):
    """
    Make a thresholded copy of `mesh` where:
      - labels outside the mask are set to 0
      - probs outside the mask are set to 0
      - percent scalar also masked (0 outside)
    Threshold can be fraction (0.8) or percent (80).
    If subset=True, returns a mesh containing only vertices/cells >= threshold.
    """
    thr = threshold / 100.0 if threshold > 1.0 else float(threshold)

    m = mesh.copy(deep=True)
    pct = np.asarray(m.point_data[percent_scalar], dtype=float)
    mask = pct >= thr

    labels = np.asarray(m.point_data[label_in], dtype=np.int32)
    probs  = np.asarray(m.point_data[prob_in], dtype=float)

    # Masked outputs
    m.point_data[label_out]   = np.where(mask, labels, 0).astype(np.int32)
    m.point_data[prob_out]    = np.where(mask, probs,  0).astype(np.float32)
    m.point_data[percent_out] = np.where(mask, pct,    0).astype(np.float32)

    # Optionally reduce to only >= threshold vertices/cells
    if subset:
        try:
            # Use the percent scalar for thresholding (point preference)
            m = m.threshold(thr, scalars=percent_scalar, preference='point')
        except Exception:
            # Fallback: if threshold fails (e.g., mesh type), just keep masked arrays
            pass

    # Trim extra arrays if requested
    if keep_only is not None:
        keep = set(keep_only) | {label_out, prob_out, percent_out, percent_scalar}
        data = {k: m.point_data[k].copy() for k in list(m.point_data.keys()) if k in keep}
        m.point_data.clear()
        for k, v in data.items():
            m.point_data[k] = v

    return m

def prepare_mesh_for_interpolation(m):
    # Ensure VTK type
    if not isinstance(m.mesh, pv.PolyData):
        m.mesh = pv.PolyData(m.mesh)

    # 1) Initialize scalar cache used by m.scalar_names (avoids _scalar_names error)
    names = set(m.point_data.keys())
    cd = m.mesh.GetCellData()
    for i in range(cd.GetNumberOfArrays()):
        arr = cd.GetArray(i)
        if arr is not None and arr.GetName():
            names.add(arr.GetName())
    m._scalar_names = names  # <-- key line

    # 2) Ensure a cell index array exists (what add_cell_idx ultimately wants)
    if cd.HasArray('cell_idx') != 1:
        n_cells = m.mesh.GetNumberOfCells()
        arr = numpy_to_vtk(np.arange(n_cells, dtype=np.int32))
        arr.SetName('cell_idx')
        cd.AddArray(arr)

    # 3) Keep YOUR names; just enforce dtypes
    if 'most_likely_label' in m.point_data:
        m.point_data['most_likely_label'] = np.asarray(m.point_data['most_likely_label'], dtype=np.int32)
    if 'most_likely_prob' in m.point_data:
        m.point_data['most_likely_prob'] = np.asarray(m.point_data['most_likely_prob'], dtype=np.float32)
    if 'percent_people_with_cartilage' in m.point_data:
        m.point_data['percent_people_with_cartilage'] = np.asarray(m.point_data['percent_people_with_cartilage'], dtype=np.float32)

    return m


def build_og_to_new(label_ids, updated_labels=(11, 12, 13, 14, 15)):
    """
    Make a mapping {orig_label -> new_label}.
    Expects `label_ids` to be five ints like [10..14] or [11..15].
    """
    orig = sorted(int(x) for x in set(label_ids))
    if len(orig) != len(updated_labels):
        raise ValueError(f"Expected {len(updated_labels)} labels, got {len(orig)}: {orig}")
    return dict(zip(orig, updated_labels))

# Settings

In [ ]:
bone = 'femur'
thr = 90

# LOAD MODEL

In [4]:
EPOCH = 2000
if bone == 'femur':
    path_model_folder = '/dataNAS/people/aagatti/projects/nsm_femur/training_run_files/experiment_results/647_nsm_femur_v0.0.1'
elif bone == 'tibia':
    path_model_folder = '/dataNAS/people/aagatti/projects/nsm_femur/training_run_files/experiment_results/650_nsm_tibia_v0.0.1'
elif bone == 'patella':
    path_model_folder = '/dataNAS/people/aagatti/projects/nsm_femur/training_run_files/experiment_results/648_nsm_patella_v0.0.1'

path_config = os.path.join(path_model_folder, 'model_params_config.json')
path_model_state = os.path.join(path_model_folder, 'model', f'{EPOCH}.pth')

with open(path_config, 'r') as f:
    config = json.load(f)

decoder = load_model(config, path_model_state, model_type='triplanar')

# Read in healthy NSM models

In [ ]:
main = f'/dataNAS/people/kenziew/projects/pymskt-test/mw_testing/experiments/fill_holes_{bone}_MOAKS0/avg_all_healthy_data_{bone}_MOAKS0/final_healthy_NSM_labels_{bone}_MOAKS0/'
final_mesh = BoneMesh(pv.read(os.path.join(main, f'mean_healthy_{bone}_505MOAKS0.vtk')))

# Create healthy mesh with threshold
lbl_name = f"label_{int(round(thr if thr > 1 else thr*100))}"
healthy_mesh_thr = masked_healthy_mesh(final_mesh, thr, label_out=lbl_name)
healthy_mesh_prepped = prepare_mesh_for_interpolation(final_mesh.copy(deep=True))
print("has _scalar_names:", hasattr(healthy_mesh_prepped, "_scalar_names"))  # should be True

# read healthy latent
avg_healthy_latent_np = np.load(
    f'/dataNAS/people/kenziew/projects/pymskt-test/mw_testing/experiments/fill_holes_{bone}_MOAKS0/avg_all_healthy_data_{bone}_MOAKS0/avg_healthy_latent_np_{bone}_MOAKS0.pkl',
    allow_pickle=True)['avg_healthy_latent']
avg_healthy_latent = torch.Tensor(avg_healthy_latent_np).to('cuda')


has _scalar_names: True


# Read in df with data

In [ ]:
df = pd.read_csv('/dataNAS/people/kenziew/projects/pymskt-test/mw_testing/experiments/OAI_FNIH_ORS2026/final_ORS_fnih_subjects_list_with_paths_and_labels.csv')

In [13]:
all_rows = []
for i in range(0, 1):
    sub_name = df.loc[i, 'sub']
    side = df.loc[i, 'limb']
    school = df.loc[i, 'school']
    timepoint = df.loc[i, 'timepoint']
    print(sub_name, side, school, timepoint)

    # get surface mesh paths 
    sub_surface_path = df.loc[i, 'femur_surf_mesh']
    sub_nsm_mesh_path = df.loc[i, 'femur_nsm_mesh']
    sub_recon_results_path = df.loc[i, 'sub_recon_results']
    sub_latent_path = df.loc[i, 'femur_latent']
    label_ids_all = df.loc[i, 'label_ids']
    label_ids_all = ast.literal_eval(label_ids_all) 
    label_ids = [label for label in label_ids_all if label > 9]

    og_mesh = BoneMesh(pv.read(sub_surface_path))
    sub_latent = torch.Tensor(np.load(sub_latent_path).squeeze()).to('cuda')

    if school == 'Stanford' and timepoint == 'V00':
        # Transfer scalars from original to recon mesh (both in geometric space)
        recon_nsm_femur_mesh = BoneMesh(pv.read(sub_nsm_mesh_path)) 
        recon_nsm_femur_mesh_with_scalars = recon_nsm_femur_mesh.copy()
        recon_nsm_femur_mesh_with_scalars.resample_surface(clusters=20_000)
        recon_nsm_femur_mesh_with_scalars.copy_scalars_from_other_mesh_to_current(
            og_mesh
        )

        # Perform ICP transform on the RECON mesh that now has the scalars so 
        # RECON and mean healthy bone are in same normalized NSM space
        with open(sub_recon_results_path, 'r') as f:
            icp_data = json.load(f)
            icp_transform = np.array(icp_data['icp_transform'])  # make it a NumPy array
            scale = icp_data['scale']
            center = icp_data['center']

        mesh_copy = recon_nsm_femur_mesh_with_scalars.copy()
        points_ = mesh_copy.point_coords
        points_ = np.concatenate([points_, np.ones((points_.shape[0], 1))], axis=1)
        points_ = points_ @ icp_transform.T
        points_ = points_[:, :3]
        mesh_copy.point_coords = points_

        recon_final = BoneMesh(mesh_copy.copy(deep=True))
    else:
        recon_final = BoneMesh(pv.read(sub_nsm_mesh_path)) 

    # at this point we have sub_latent, icp transform, and recon_final in nsm space with scalars
    
    # interpolate healthy mesh to subject
    interpolated_healthy_mesh = interpolate_mesh(
        decoder,
        avg_healthy_latent.cpu().numpy().astype(float).squeeze(),
        sub_latent.cpu().numpy().astype(float).squeeze(),
        mesh=healthy_mesh_prepped,      
        adaptive=False
    )

    #transform interpolated_healthy_mesh to origginal space
    with open(sub_recon_results_path, 'r') as f:
        alignment = json.load(f)
    transform = np.array(alignment['icp_transform'])
    inverse_transform = np.linalg.inv(transform)
    mesh_copy = interpolated_healthy_mesh.copy()
    points_ = mesh_copy.point_coords
    points_ = np.concatenate([points_, np.ones((points_.shape[0], 1))], axis=1)
    points_ = points_ @ inverse_transform.T
    points_ = points_[:, :3]
    mesh_copy.point_coords = points_

    lbl_name = f"label_{int(round(thr if thr > 1 else thr*100))}"
    healthy_mesh_thr = masked_healthy_mesh(mesh_copy, thr, label_out=lbl_name)

    # transfer labels from healthy mesh to interpolated subject mesh
    og_mesh_updated = og_mesh.copy(deep=True)
    og_mesh_updated.copy_scalars_from_other_mesh_to_current(
        healthy_mesh_thr,
        orig_scalars_name=[lbl_name],
        new_scalars_name=[lbl_name]
    )

    og_to_new = {10:11, 11:12, 12:13, 13:14, 14:15}
    break

#     df_thickness = compute_thickness_by_label_single_mesh(
#         mesh=og_mesh_updated,
#         label_map=build_og_to_new(label_ids),
#         orig_label_scalar='labels',
#         updated_label_scalar=lbl_name,
#         thickness_scalar='thickness (mm)'
#     )

#     # attach metadata columns (broadcast across rows)
#     meta = {
#         'sub': sub_name,
#         'limb': side,
#         'school': school,
#         'timepoint': timepoint,
#         'femur_surf_mesh': sub_surface_path,
#         'femur_nsm_mesh': sub_nsm_mesh_path,
#         'sub_recon_results': sub_recon_results_path,
#         'femur_latent': sub_latent_path,
#         'label_ids': json.dumps(label_ids),              # keep as JSON string for CSV safety
#         'label_map': json.dumps(og_to_new),
#         'updated_label_scalar': lbl_name,
#         'thr': thr,
#     }
#     df_thickness = df_thickness.assign(**meta)
#     all_rows.append(df_thickness)
         
# # ---- after loop: combine and save
# df_thickness_all = pd.concat(all_rows, ignore_index=True)

# # choose your output path
# df_thickness_all.to_csv('ORS_thickness_by_label_FNIH_subjects.csv', index=False)

9001695 RIGHT Staple V00


In [14]:
view(geometries=og_mesh_updated)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [15]:
labels = og_mesh_updated.point_data['labels']
labels_new = og_mesh_updated.point_data[lbl_name]
for label_id, new_label_id in og_to_new.items():
    mask = (labels == label_id).astype(float)
    mask_new = (labels_new == new_label_id).astype(float)
    print(f"label {label_id} mask: {mask.sum()}")
    print(f"label {new_label_id} mask: {mask_new.sum()}")
    print(f"label {label_id} mask: {mask.sum()}")
    print(f"label {new_label_id} mask: {mask_new.sum()}")
    print(f"label {label_id} mask: {mask.sum()}")
    print(f"label {new_label_id} mask: {mask_new.sum()}")
    break

label 10 mask: 993.0
label 11 mask: 1026.0
label 10 mask: 993.0
label 11 mask: 1026.0
label 10 mask: 993.0
label 11 mask: 1026.0


In [28]:
def compute_thickness_by_label_single_mesh(
    mesh,
    label_ids=None,                 # optional: if no mapping given, compare same→same for these IDs
    label_map=None,                 # dict like {orig_label: new_label}
    p=None,                         # any tag you want to carry through (e.g., threshold)
    orig_label_scalar='labels',     # ORIGINAL labels = area covered by cartilage (cAB)
    updated_label_scalar='label_50',# UPDATED labels = total subchondral bone area (tAB)
    thickness_scalar='thickness (mm)',
    zero_tol=1e-6,                  # thickness <= zero_tol ⇒ "denuded" vertex
    write_per_label_arrays=False
):
    """
    Compare thickness and surface *areas (cm² only)* per label between original labels and mapped new labels on a SINGLE mesh.

    Areas (cm²):
      - area_covered_cm2 : ORIGINAL labels (cAB), via triangle mask (all 3 verts in label)
      - area_total_cm2   : UPDATED  labels (tAB), via triangle mask (all 3 verts in label)
      - area_denuded_cm2 : UPDATED  labels, vertex-based: sum per-vertex areas where thickness <= zero_tol
                           (per-vertex area weights formed by splitting each triangle area equally among its 3 vertices)

    Thickness (mm):
      - mean_thickness_orig : mean over ORIGINAL-label vertices
      - mean_thickness_new  : mean over UPDATED-label vertices
    """
    import numpy as np
    import pandas as pd

    # --- sanity checks for point data existence
    keys = mesh.point_data.keys()
    for name in (orig_label_scalar, updated_label_scalar, thickness_scalar):
        if name not in keys:
            raise KeyError(f"Missing '{name}' on mesh. Available point_data: {list(keys)[:15]}...")

    # --- arrays (point data) for thickness stats (point-based)
    labels_orig_pts = np.asarray(mesh.point_data[orig_label_scalar], dtype=np.int32)
    labels_new_pts  = np.asarray(mesh.point_data[updated_label_scalar], dtype=np.int32)
    thick           = np.asarray(mesh.point_data[thickness_scalar], dtype=float)
    n_points        = int(mesh.n_points)

    # --- build mapping
    if label_map is None:
        if label_ids is None:
            label_ids = np.unique(labels_orig_pts).tolist()
        label_map = {int(l): int(l) for l in label_ids}
    else:
        label_map = {int(k): int(v) for k, v in label_map.items()}

    # ====== AREA PREP (triangle-by-triangle, mask-based) ======
    surf = mesh.extract_surface().triangulate()
    surf = surf.compute_cell_sizes(length=False, area=True, volume=False)  # adds 'Area' (mm²) to cell_data
    tri_areas = np.asarray(surf.cell_data['Area'], dtype=float)            # (n_tris,)
    faces = surf.faces.reshape(-1, 4)[:, 1:]                               # (n_tris, 3) vertex indices

    # Ensure needed arrays exist on the surface points; if not, remap via vtkOriginalPointIds
    def _ensure_on_surface(arr_name, src_mesh_array):
        if arr_name in surf.point_data:
            return np.asarray(surf.point_data[arr_name])
        if 'vtkOriginalPointIds' in surf.point_data:
            orig_ids = np.asarray(surf.point_data['vtkOriginalPointIds'])
            vals = np.asarray(src_mesh_array)[orig_ids]
            surf.point_data[arr_name] = vals
            return vals
        vals = np.asarray(src_mesh_array)[:surf.n_points]
        surf.point_data[arr_name] = vals
        return vals

    surf_labels_orig = _ensure_on_surface(orig_label_scalar, mesh.point_data[orig_label_scalar])
    surf_labels_new  = _ensure_on_surface(updated_label_scalar, mesh.point_data[updated_label_scalar])
    surf_thick       = _ensure_on_surface(thickness_scalar,     mesh.point_data[thickness_scalar])

    def area_from_point_mask(point_mask_bool):
        """Sum areas of triangles whose 3 vertices all satisfy the point mask (returns mm²)."""
        tri_ok = point_mask_bool[faces].all(axis=1)
        return float(tri_areas[tri_ok].sum())

    # Per-vertex area weights (each triangle's area split equally to its 3 vertices), mm²
    vtx_area = np.zeros(surf.n_points, dtype=float)
    np.add.at(vtx_area, faces.ravel(), np.repeat(tri_areas / 3.0, 3))

    # ====== ROW ACCUMULATOR ======
    rows = {
        'orig_label': [],
        'new_label': [],
        'mean_thickness_orig': [],
        'mean_thickness_new': [],
        'n_points_orig': [],
        'n_points_new': [],
        # cm² areas only
        'area_total_cm2': [],        # tAB (updated labels)
        'area_covered_cm2': [],      # cAB (original labels)
        'area_denuded_cm2': [],      # NSM definition: zero-thickness vertices area on updated labels

        # bookkeeping
        'p': [],
        'orig_label_scalar': [],
        'updated_label_scalar': [],
    }

    # ====== PER-LABEL LOOP ======
    for orig_label, new_label in label_map.items():
        # --- thickness on POINTS (original mesh space)
        mask_o_pts = (labels_orig_pts == orig_label)
        mask_n_pts = (labels_new_pts  == new_label)

        vals_o = thick[mask_o_pts]
        vals_n = thick[mask_n_pts]

        n_o = int(mask_o_pts.sum())
        n_n = int(mask_n_pts.sum())

        mean_o = np.nan if n_o == 0 else float(np.nanmean(vals_o))
        mean_n = np.nan if n_n == 0 else float(np.nanmean(vals_n))

        # --- AREAS (mm² → cm²) via SURFACE triangle masks
        area_o_mm2 = area_from_point_mask(surf_labels_orig == orig_label)  # cAB
        area_n_mm2 = area_from_point_mask(surf_labels_new  == new_label)   # tAB
        area_total_cm2   = area_n_mm2 / 100.0
        area_covered_cm2 = area_o_mm2 / 100.0
        pct_cov = (100.0 * area_covered_cm2 / area_total_cm2) if area_total_cm2 > 0 else np.nan

        # --- DENUDED AREA (UPDATED label, zero-thickness VERTICES) on SURFACE
        new_mask_surf   = (surf_labels_new == new_label)
        zero_mask_surf  = new_mask_surf & np.isfinite(surf_thick) & (surf_thick <= float(zero_tol))
        area_denuded_cm2 = float(vtx_area[zero_mask_surf].sum()) / 100.0

        # counts on SURFACE (use surface denominator for consistency)
        n_zero_surf = int(zero_mask_surf.sum())
        n_new_surf  = int(new_mask_surf.sum())
        zero_frac   = (n_zero_surf / n_new_surf) if n_new_surf > 0 else np.nan

        # --- record
        rows['orig_label'].append(int(orig_label))
        rows['new_label'].append(int(new_label))
        rows['mean_thickness_orig'].append(mean_o)
        rows['mean_thickness_new'].append(mean_n)
        rows['n_points_orig'].append(n_o)
        rows['n_points_new'].append(n_n)

        rows['area_total_cm2'].append(area_total_cm2)
        rows['area_covered_cm2'].append(area_covered_cm2)
        rows['area_denuded_cm2'].append(area_denuded_cm2)

        rows['p'].append(p)
        rows['orig_label_scalar'].append(orig_label_scalar)
        rows['updated_label_scalar'].append(updated_label_scalar)

        if write_per_label_arrays:
            arr_o = np.full(n_points, np.nan, dtype=float); arr_o[mask_o_pts] = thick[mask_o_pts]
            arr_n = np.full(n_points, np.nan, dtype=float); arr_n[mask_n_pts] = thick[mask_n_pts]
            mesh.point_data[f'thickness_orig_{orig_label} (mm)'] = arr_o
            mesh.point_data[f'thickness_{updated_label_scalar}_{new_label} (mm)'] = arr_n

    return pd.DataFrame(rows)


In [29]:
df_thickness = compute_thickness_by_label_single_mesh(
    mesh=og_mesh_updated,
    label_map=build_og_to_new(label_ids),
    orig_label_scalar='labels',
    updated_label_scalar=lbl_name,
    thickness_scalar='thickness (mm)'
)
df_thickness

,orig_label,new_label,mean_thickness_orig,mean_thickness_new,n_points_orig,n_points_new,area_total_cm2,area_covered_cm2,area_denuded_cm2,p,orig_label_scalar,updated_label_scalar
0,10,11,2.641065,2.667876,993,1026,19.320133,18.586432,0.437066,None,labels,label_90
1,11,12,2.011454,1.937687,405,324,5.605411,7.159269,0.246170,None,labels,label_90
2,12,13,2.073925,2.136364,431,328,5.782023,7.551403,0.063343,None,labels,label_90
3,13,14,2.118853,2.122831,567,520,9.626410,10.493435,0.252670,None,labels,label_90
4,14,15,1.667954,1.557869,528,452,8.052806,9.312699,0.657737,None,labels,label_90
